In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [3]:
# preprocess data 
# Drop irrevalent features
df = df.drop(columns=['RowNumber','CustomerId','Surname'],axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [8]:
df.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [4]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [5]:
lable_encoder_gender = LabelEncoder()
df['Gender'] = lable_encoder_gender.fit_transform(df['Gender'])
df['Gender'].unique()

array([0, 1])

In [11]:
# lable_encoder_geo = LabelEncoder()
# df['Geography'] = lable_encoder_geo.fit_transform(df['Geography'])
# df.head()
# df['Geography'].unique()
#  2 can be considered more than 1

In [6]:
from sklearn.preprocessing import OneHotEncoder
ohe_geo = OneHotEncoder(sparse_output=False)

In [7]:
# Fit-transform produces a numpy array; keep row alignment by using df.index when
# constructing the DataFrame below.
geo_encoded = ohe_geo.fit_transform(df[['Geography']])
geo_encoded


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [8]:
# Create the one-hot DataFrame using the original DataFrame's index to avoid
# alignment issues (which cause NaNs when concatenating).
geo_df = pd.DataFrame(geo_encoded,
                      columns=ohe_geo.get_feature_names_out(['Geography']),
                      index=df.index)
geo_df.head()


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [9]:
# combine columns 
df = pd.concat([df.drop(columns=['Geography'],axis=1),geo_df],axis=1)

In [10]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
# save the encoders
with open('label_encoder_geo.pkl','wb') as file:
    pickle.dump(ohe_geo,file) 
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(lable_encoder_gender,file) 

In [12]:
# Divide dataset into dependend and independent feature
X = df.drop(columns=['Exited'],axis=1)
y = df['Exited']
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0


In [13]:
# divide in train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
# Scaling the independent feature
standardise = StandardScaler()
X_train_standard = standardise.fit_transform(X_train)
X_test_standard = standardise.transform(X_test)
X_train_standard

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]], shape=(8000, 12))

In [15]:
with open('standard_scaler.pkl','wb') as file:
    pickle.dump(standardise,file)

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
# in tf we call ANN as sequential Model
# trainable parameter = total no. of weight+total no of bias

In [17]:
# Build ANN
model = Sequential(
    [
        Dense(64,activation='relu',input_shape = (X_train.shape[1],)), #HL1 connected to input layer
        Dense(32,activation='relu'), #HL2
        Dense(1,activation='sigmoid') # O/P Layer
    ]
)
model.summary()

c:\Users\Test\Desktop\Flask\ML\Deep Learning\ANN_Classification\venv\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()
# we can use this

In [19]:
# compile model
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [20]:
# setup Tensorboard to visualize the logs
log_dir = 'logs/fit'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [21]:
# setup early stopping if we have selected 100 epoches but model got trained in 20 
# epoches only and no major changes are seen in loss value then it will stop furthur epoches
early_stopping_callback = EarlyStopping(monitor ='val_loss',patience=10,restore_best_weights=True)

In [22]:
# Train Model
history = model.fit(X_train_standard,y_train,
                    validation_data=(X_test_standard,y_test),
                    epochs=100,
                    callbacks = [tensorflow_callback,early_stopping_callback]
                    )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8263 - loss: 0.4108 - val_accuracy: 0.8610 - val_loss: 0.3516
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8263 - loss: 0.4108 - val_accuracy: 0.8610 - val_loss: 0.3516
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8550 - loss: 0.3556 - val_accuracy: 0.8605 - val_loss: 0.3484
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8550 - loss: 0.3556 - val_accuracy: 0.8605 - val_loss: 0.3484
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8550 - loss: 0.3498 - val_accuracy: 0.8580 - val_loss: 0.3476
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8550 - loss: 0.3498 - val_accuracy: 0.8580 - val_loss: 0.3476
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8584 - loss: 0.3433 - val_accuracy: 0.8635 - val_loss: 0.3520
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8584 - loss: 0.3433 - val_accu

In [23]:
model.save('model.h5')

In [25]:
# load tensorboard Extension
%load_ext tensorboard

In [27]:
%tensorboard --logdir logs/fit20251129-225543

In [ ]:
#